In [53]:
from TableExtract import HTMLTableParser
import pandas as pd
from bs4 import BeautifulSoup
import requests

import warnings
warnings.filterwarnings('ignore')

In [62]:
#url = 'http://www.cfbstats.com/2017/team/8/passing/defense/gamelog.html'
#url = 'http://www.cfbstats.com/2017/team/31/interception/gamelog.html'
url = 'http://www.cfbstats.com/2017/team/234/passing/offense/gamelog.html'

In [68]:
hp = HTMLTableParser()
df = hp.parse_url(url)[0][1]
df

,Date,Opponent,Surface,Result,Att,Comp,Pct.,Yards,Yards/Att,TD,Int,Rating
0,09/02/17,+ 1 Alabama,Turf,L 7-24,33.0,19.0,57.6,210.0,6.4,1.00,2.0,108.92
1,09/23/17,23 North Carolina St.,Grass,L 21-27,39.0,22.0,56.4,278.0,7.1,1.00,0.0,124.75
2,09/30/17,@ Wake Forest,Turf,W 26-19,21.0,11.0,52.4,121.0,5.8,1.00,0.0,116.49
3,10/07/17,13 Miami (Fla.),Grass,L 20-24,28.0,17.0,60.7,203.0,7.3,2.00,2.0,130.89
4,10/14/17,@ Duke,Grass,W 17-10,21.0,18.0,85.7,197.0,9.4,1.00,2.0,161.17
5,10/21/17,Louisville,Grass,L 28-31,28.0,16.0,57.1,248.0,8.9,2.00,2.0,140.82
6,10/27/17,@ Boston College,Turf,L 3-35,27.0,12.0,44.4,149.0,5.5,0.00,1.0,83.39
7,11/04/17,Syracuse,Grass,W 27-24,19.0,12.0,63.2,136.0,7.2,1.00,0.0,140.66
8,11/11/17,@ 4 Clemson,Grass,L 14-31,32.0,13.0,40.6,208.0,6.5,1.00,1.0,99.29
9,11/18/17,Delaware St.,Grass,W 77-6,24.0,18.0,75.0,236.0,9.8,4.00,0.0,212.60


In [11]:
#FUNC: Drop last two rows
def dropTail(table):
    new_table = table.drop(table.tail(2).index)
    return new_table

In [14]:
#FUNC: Add game locations
def homeAway(table):
    game_locations = []

    for row in table['Opponent']:
        if '@' in row:
            game_locations.append('away')
        elif '+' in row:
            game_locations.append('neutral')
        else:
            game_locations.append('home')

    table['Location'] = game_locations
    return table

In [30]:
#FUNC: Update opponent names
def opponentName(tableURL, table):
    response = requests.get(tableURL)
    soup = BeautifulSoup(response.text, 'lxml')
    rows = soup.select('td[class="opponent"]')
    
    team_names = []
    
    for row in rows:
        try:
            team_names.append(row.find('a').text)
        except:
            team_names.append('FCS')
    
    table['Opponent'] = team_names
    return table

In [69]:
df = dropTail(df)
df = homeAway(df)
df = opponentName(url, df)
df.head()

,Date,Opponent,Surface,Result,Att,Comp,Pct.,Yards,Yards/Att,TD,Int,Rating,Location
0,09/02/17,Alabama,Turf,L 7-24,33.0,19.0,57.6,210.0,6.4,1.0,2.0,108.92,neutral
1,09/23/17,North Carolina St.,Grass,L 21-27,39.0,22.0,56.4,278.0,7.1,1.0,0.0,124.75,home
2,09/30/17,Wake Forest,Turf,W 26-19,21.0,11.0,52.4,121.0,5.8,1.0,0.0,116.49,away
3,10/07/17,Miami (Fla.),Grass,L 20-24,28.0,17.0,60.7,203.0,7.3,2.0,2.0,130.89,home
4,10/14/17,Duke,Grass,W 17-10,21.0,18.0,85.7,197.0,9.4,1.0,2.0,161.17,away


In [60]:
#FUNC: Extract offensive passing
def extractOffensivePassing(table):
    table[['Result','Score']] = df['Result'].str.split(' ',expand=True)
    
    table = table[['Date', 'Opponent', 'Result', 'Score', 'Att', 'Yards', 'Int', 'Location']]
    table[['Att', 'Yards', 'Int']] = table[['Att', 'Yards', 'Int']].astype('int64')
    table = table.rename(columns={'Att':'Off Pass Att', 'Yards':'Off Pass Yards', 'Int':'Off Int'})
    
    return table

In [70]:
df = extractOffensivePassing(df)
df.head()

,Date,Opponent,Result,Score,Off Pass Att,Off Pass Yards,Off Int,Location
0,09/02/17,Alabama,L,7-24,33,210,2,neutral
1,09/23/17,North Carolina St.,L,21-27,39,278,0,home
2,09/30/17,Wake Forest,W,26-19,21,121,0,away
3,10/07/17,Miami (Fla.),L,20-24,28,203,2,home
4,10/14/17,Duke,W,17-10,21,197,2,away


In [71]:
#FUNC: Extract defensive passing
def extractDefensivePassing(table):
    table = table[['Att', 'Yards']]
    table[['Att', 'Yards']] = table[['Att', 'Yards']].astype('int64')
    table = table.rename(columns={'Att':'Def Pass Att', 'Yards':'Def Pass Yards'})
    
    return table

In [72]:
#FUNC: Extract offensive rushing
def extractOffensiveRushing(table):
    table = table[['Att', 'Yards']]
    table[['Att', 'Yards']] = table[['Att', 'Yards']].astype('int64')
    table = table.rename(columns={'Att':'Off Rush Att', 'Yards':'Off Rush Yards'})
    
    return table

In [ ]:
#FUNC: Extract defensive rushing
def extractDefensiveRushing(table):
    table = table[['Att', 'Yards']]
    table[['Att', 'Yards']] = table[['Att', 'Yards']].astype('int64')
    table = table.rename(columns={'Att':'Def Rush Att', 'Yards':'Def Rush Yards'})
    
    return table

In [32]:
#------------------------------------------#
#--------------Concat example--------------#
#------------------------------------------#
test1 = {'A':[1,2], 'B':[3,4]}
test2 = {'C':[5,6], 'D':[7,8]}

tf1 = pd.DataFrame(test1)
tf2 = pd.DataFrame(test2)

frames = [tf1, tf2]
result = pd.concat(frames, axis=1)
result